# Python AI Chess Game


## Importing Libraries

In [ ]:
import chess
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython.display import clear_output
from collections import OrderedDict
from operator import itemgetter

## Functions

In [ ]:
def predict(df, model):
    predictions = []
    for _, row in df.iterrows():
        example = tf.train.Example()
        for colName, dtype in zip(df.columns, df.dtypes):
            value = row[colName]
            if dtype == object:
                value = value.encode('utf-8')
                example.features.feature[colName].bytes_list.value.extend([value])
            elif dtype == float:
                example.features.feature[colName].float_list.value.extend([value])
            elif dtype == int:
                example.features.feature[colName].int64_list.value.extend([value])
        serialized_example = example.SerializeToString()
        input_data = tf.constant([serialized_example])
        output = model.signatures['predict'](examples=input_data)
        predictions.append(output)
    return predictions

def createMovesDataframe(board):
    squareFeatures = [str(board.piece_at(i)) for i in chess.SQUARES]
    data = []
    moves = list(board.legal_moves)
    for move in moves:
        fromSquare = np.zeros(64)
        toSquare = np.zeros(64)
        fromSquare[move.from_square] = 1
        toSquare[move.to_square] = 1
        row = squareFeatures + fromSquare.tolist() + toSquare.tolist()
        data.append(row)

    squareNames = chess.SQUARE_NAMES
    moveFromFeatures = ['from_' + square for square in squareNames]
    moveToFeatures = ['to_' + square for square in squareNames]

    columns = squareNames + moveFromFeatures + moveToFeatures

    df = pd.DataFrame(data=data, columns=columns)
    df[moveFromFeatures + moveToFeatures] = df[moveFromFeatures + moveToFeatures].astype(float)

    return df

def getPieceValue(piece, square):
    x, y = divmod(square, 8)

    if AI_black:
        white = -1
        black = 1
    else:
        white = 1
        black = -1

    if piece == 'None':
        return 0
    elif piece == 'P':
        return white * (10 + pawnWhiteEvaluations[x][y])
    elif piece == 'N':
        return white * (30 + knightWhiteEvaluations[x][y])
    elif piece == 'B':
        return white * (30 + bishopWhiteEvaluations[x][y])
    elif piece == 'R':
        return white * (50 + rookWhiteEvaluations[x][y])
    elif piece == 'Q':
        return white * (90 + queenWhiteEvaluations[x][y])
    elif piece == 'K':
        return white * (900 + kingWhiteEvaluations[x][y])
    elif piece == 'p':
        return black * (10 + pawnBlackEvaluations[x][y])
    elif piece == 'n':
        return black * (30 + knightBlackEvaluations[x][y])
    elif piece == 'b':
        return black * (30 + bishopBlackEvaluations[x][y])
    elif piece == 'r':
        return black * (50 + rookBlackEvaluations[x][y])
    elif piece == 'q':
        return black * (90 + queenBlackEvaluations[x][y])
    elif piece == 'k':
        return black * (900 + kingBlackEvaluations[x][y])

def minimaxAlphaBetaPruning(depth, board, model):
    def minimaxAux(depth, board, alpha, beta, guard):
        if depth == 0:
            boardEvaluation = sum(getPieceValue(str(board.piece_at(i)), i) for i in chess.SQUARES)
            return -boardEvaluation

        if depth > 3:
            movesList = list(board.legal_moves)
            df = createMovesDataframe(board)
            predictions = predict(df, model)
            goodMoveProbabilities = [tf.make_ndarray(tf.make_tensor_proto(i['probabilities']))[0][1] for i in predictions]
            movesAndProbs = list(zip(movesList, goodMoveProbabilities))
            movesAndProbs.sort(key=lambda x: x[1], reverse=True)
            legalMoves = [move for move, _ in movesAndProbs[:int(len(movesAndProbs) * 0.5)]]
        else:
            legalMoves = list(board.legal_moves)

        bestMove = -9999 if guard else 9999
        bestMoveCalculated = None

        for move in legalMoves:
            board.push(move)
            value = minimaxAux(depth - 1, board, alpha, beta, not guard)
            board.pop()

            if guard:
                bestMove = max(bestMove, value)
                alpha = max(alpha, bestMove)
                if beta <= alpha:
                    break
            else:
                bestMove = min(bestMove, value)
                beta = min(beta, bestMove)
                if beta <= alpha:
                    break

        return bestMove

    movesList = list(board.legal_moves)
    df = createMovesDataframe(board)
    predictions = predict(df, model)
    goodMoveProbabilities = [tf.make_ndarray(tf.make_tensor_proto(i['probabilities']))[0][1] for i in predictions]
    movesAndProbs = list(zip(movesList, goodMoveProbabilities))
    movesAndProbs.sort(key=lambda x: x[1], reverse=True)
    legalMoves = [move for move, _ in movesAndProbs[:int(len(movesAndProbs) * 0.5)]]

    bestMove = -9999
    bestMoveCalculated = None
    guard = True

    for move in legalMoves:
        board.push(move)
        value = minimaxAux(depth - 1, board, -10000, 10000, not guard)
        board.pop()

        if value >= bestMove:
            bestMove = value
            bestMoveCalculated = move

    return bestMoveCalculated

def checkmate(move, board):
    fen = board.fen()
    boardCheckmate = chess.Board(fen)
    boardCheckmate.push(move)
    return boardCheckmate.is_checkmate()

def startGame(guard, board):
    if board.is_stalemate():
        clear_output()
        display(board)
        print("Stalemate!")
        return
    else:
        if guard:
            if not board.is_checkmate():
                clear_output()
                display(board)
                inputMove = input("Enter your move: ")
                move = chess.Move.from_uci(inputMove)
                if move not in board.legal_moves:
                    return startGame(guard, board)

                board.push(move)
                return startGame(not guard, board)
            else:
                clear_output()
                display(board)
                print("AI wins!")
                return
        else:
            if not board.is_checkmate():
                clear_output()
                display(board)
                for move in board.legal_moves:
                    if checkmate(move, board):
                        board.push(move)
                        clear_output()
                        display(board)
                        print("AI wins!")
                        return

                numberOfMoves = len(list(board.legal_moves))

                if numberOfMoves > 30:
                    board.push(minimaxAlphaBetaPruning(4, board, model))
                elif 10 < numberOfMoves <= 30:
                    board.push(minimaxAlphaBetaPruning(5, board, model))
                else:
                    board.push(minimaxAlphaBetaPruning(7, board, model))

                return startGame(not guard, board)

            else:
                clear_output()
                display(board)
                print("User wins!")
                return

## Piece Initialization Values

In [ ]:
pawnWhiteEvaluations = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
                            [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0],
                            [1.0, 1.0, 2.0, 3.0, 3.0, 2.0, 1.0, 1.0],
                            [0.5, 0.5, 1.0, 2.5, 2.5, 1.0, 0.5, 0.5],
                            [0.0, 0.0, 0.0, 2.0, 2.0, 0.0, 0.0, 0.0],
                            [0.5, -0.5, -1.0, 0.0, 0.0, -1.0, -0.5, 0.5],
                            [0.5, 1.0, 1.0, -2.0, -2.0, 1.0, 1.0, 0.5],
                            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]], np.double)

pawnBlackEvaluations = pawnWhiteEvaluations[::-1]

knightWhiteEvaluations = np.array([[-5.0, -4.0, -3.0, -3.0, -3.0, -3.0, -4.0, -5.0],
                              [-4.0, -2.0, 0.0, 0.0, 0.0, 0.0, -2.0, -4.0],
                              [-3.0, 0.0, 1.0, 1.5, 1.5, 1.0, 0.0, -3.0],
                              [-3.0, 0.5, 1.5, 2.0, 2.0, 1.5, 0.5, -3.0],
                              [-3.0, 0.0, 1.5, 2.0, 2.0, 1.5, 0.0, -3.0],
                              [-3.0, 0.5, 1.0, 1.5, 1.5, 1.0, 0.5, -3.0],
                              [-4.0, -2.0, 0.0, 0.5, 0.5, 0.0, -2.0, -4.0],
                              [-5.0, -4.0, -3.0, -3.0, -3.0, -3.0, -4.0, -5.0]], np.double)

knightBlackEvaluations = knightWhiteEvaluations[::-1]

bishopWhiteEvaluations = np.array([[-2.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -2.0],
                              [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0],
                              [-1.0, 0.0, 0.5, 1.0, 1.0, 0.5, 0.0, -1.0],
                              [-1.0, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, -1.0],
                              [-1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, -1.0],
                              [-1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, -1.0],
                              [-1.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.5, -1.0],
                              [-2.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -2.0]], np.double)

bishopBlackEvaluations = bishopWhiteEvaluations[::-1]

rookWhiteEvaluations = np.array([[0.0, 0.0, 0.0, 0.0, 0.0,  0.0, 0.0, 0.0],
                            [0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5],
                            [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
                            [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
                            [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
                            [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
                            [-0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.5],
                            [ 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0]], np.double)

rookBlackEvaluations = rookWhiteEvaluations[::-1]

queenWhiteEvaluations = np.array([[-2.0, -1.0, -1.0, -0.5, -0.5, -1.0, -1.0, -2.0],
                             [-1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1.0],
                             [-1.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, -1.0],
                             [-0.5, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, -0.5],
                             [0.0, 0.0, 0.5, 0.5, 0.5, 0.5, 0.0, -0.5],
                             [-1.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.0, -1.0],
                             [-1.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, -1.0],
                             [-2.0, -1.0, -1.0, -0.5, -0.5, -1.0, -1.0, -2.0]], np.double)

queenBlackEvaluations = queenWhiteEvaluations[::-1]

kingWhiteEvaluations = np.array([[-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
                            [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
                            [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
                            [-3.0, -4.0, -4.0, -5.0, -5.0, -4.0, -4.0, -3.0],
                            [-2.0, -3.0, -3.0, -4.0, -4.0, -3.0, -3.0, -2.0],
                            [-1.0, -2.0, -2.0, -2.0, -2.0, -2.0, -2.0, -1.0],
                            [2.0, 2.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0],
                            [2.0, 3.0, 1.0, 0.0, 0.0, 1.0, 3.0, 2.0]], np.double)

kingBlackEvaluations = kingWhiteEvaluations[::-1]

## Upload the AI model simulating Bobby Fischer

In [ ]:
global model
model = tf.saved_model.load("Model")

## Play vs Bobby Fischer

In [ ]:
global AI_black
board = chess.Board()
print("Welcome to AI Chess! You are going to play vs Bobby Fischer")
print("")
print("To enter a move, write in lowercase the starting cell of the chess board followed by the destination cell. Example: b1c3")
print("")
userFirst = input("Would you start? [y/n]: ")
if(userFirst == 'y'):
    AI_black = True
    startGame(True, board)
else:
    AI_black = False
    startGame(False, board)